In [1]:
import folium
from src.apikeys import MONGO
from src import process_tweets
from pymongo import MongoClient
from src.model import TweetPredictor
import pandas as pd

In [17]:
tweet_processor = process_tweets.TweetProcessor('../models/stemmed_lr.pk')


In [18]:
client = MongoClient('mongodb://{}:{}@localhost:27017'.format(MONGO.USER, MONGO.PASSWORD))

In [19]:
database = client['twitter_database']

In [20]:
database.collection_names()

['test',
 'rainbows',
 'trends_2017_10_18_9',
 'trends_2017_10_16_17',
 'trends_2017_10_17_18',
 'broncos_raiders',
 'available_trends',
 'test_method',
 'trends_2017_10_17_19',
 'patriots_panthers',
 'trends_2017_10_17_17',
 'politics',
 'rep_dem_kasich',
 'roll_tide']

In [21]:
table = database['patriots_panthers']

In [22]:
df = tweet_processor.process_database(table, topics=['patriots'])

In [23]:
df.head(10)

,text,state,date,sentiment,time_bin
0,RT @Patriots: LET'S GO. https://t.co/Ot4IjanSa1,Virginia,2017-10-01 16:54:59+00:00,0.812222,0
1,@Patriots #PatsNation #NewEnglandPatriots #Do...,Florida,2017-10-01 16:54:59+00:00,0.812222,0
2,RT @Patriots: #TFW you hear #boomtower is back...,North Carolina,2017-10-01 16:55:00+00:00,0.680129,0
3,Theres always some extra hate in me whenever t...,North Carolina,2017-10-01 16:55:00+00:00,0.341078,0
4,RT @MikeReiss: Videoboard shows Jerod Mayo ope...,Washington,2017-10-01 16:55:01+00:00,0.713299,0
5,RT @Patriots: LET'S GO. https://t.co/Ot4IjanSa1,Massachusetts,2017-10-01 16:55:02+00:00,0.812222,0
6,Watch Carolina Panthers vs New England Patriot...,Nevada,2017-10-01 16:55:03+00:00,0.820710,0
7,Carolina Panthers VS New England Patriots Live...,None,2017-10-01 16:55:04+00:00,0.820710,0
8,RT @Deacon_1971: POLL: Which POTUS loves Ameri...,Wisconsin,2017-10-01 16:55:04+00:00,0.903831,0
9,GAMETIME: Your New England Patriots look to im...,None,2017-10-01 16:55:03+00:00,0.785331,0


In [24]:
from src.visualization import visualize_sentiment
from src.visualization import visualize_count
my_map = visualize_sentiment(df)
my_map

In [25]:
my_map.save('../website/maps/patriots_sentiment.html')

In [26]:
import numpy as np
a = df.groupby('state')['sentiment'].agg(['mean','std','count'])
a['mean_dev'] = a.apply(lambda row: row['std']/np.sqrt(row['count']),axis=1)
a[['mean','mean_dev']].head(50)

,mean,mean_dev
state,,
Alabama,0.596315,0.012148
Alaska,0.653296,0.029002
Arizona,0.630595,0.010743
Arkansas,0.621362,0.027706
California,0.616172,0.005034
Colorado,0.582991,0.013405
Connecticut,0.594820,0.007782
Delaware,0.656750,0.007033
District of Columbia,0.586051,0.023732


In [27]:
my_map, avg_sentiment = visualize_count(df[df.sentiment<.3])
print(avg_sentiment)
my_map.save('../website/maps/patriots_count.html')
my_map

0.16687936484360322


In [28]:
for i in df[df.state=='Massachusetts'][['text','sentiment']].values:
    print(i[0],i[1])

RT @Patriots: LET'S GO. https://t.co/Ot4IjanSa1 0.8122216935903996
Intrigante saber como responderá la defensiva de #patriots hoy vs #panters. estan en el sótano de la NFL 0.4584312279120568
Intrigante saber como responderá la defensiva de #patriots hoy vs #panters. estan en el sótano de la NFL 0.4584312279120568
#Patriots time!!!!! #GoPats #PatriotsNation 0.5584150143745885
Moss names Brady the best QB of all time - ESPN Video #Patriots  https://t.co/DY3oKl4MLp https://t.co/7KBzd7EGxU 0.8460850151074835
RT @NEPatsNation12: GAMETIME: Your New England Patriots look to improve their record to 3-1 against the Carolina Panthers! https://t.co/dJV… 0.8175860851109278
MATT .@mattbare23 Sorry  your request came late to me : I already put out the #Patriots as my best play if they win… https://t.co/B1jHcbVGzN 0.42519891542939137
RT @Patriots: It's game day in Foxboro! https://t.co/2tQ13eG8Xk 0.6824689195859569
RT @MarkDanielsPJ: Looks like the Patriots will stand together for the anthem 0.617451

@emsfil12 @Patriots They all stood today with hand over heart. Feels good to love my team again. 0.7432979825117648
RT @BostonBlabber: Here's the #Patriots standing along sideline during the anthem today. 👏 https://t.co/8dmTNNXp1d 0.7216516829741734
Kuechly just making every tackle. #CARvsNE #Patriots 0.5863332712442763
RT @CaseyBake16: INTERCEPTION! Malcolm Butler doing Malcolm Butler things! #Patriots ball! #CARvsNE https://t.co/TpJiLhVjVB 0.741349018326968
RT @EvanDrellich: Mute the Patriots game. Turn on @WEEI, and we'll break down Red Sox-Astros. That’s two birds with one stone. But we try t… 0.5246105466905213
RT @Patriots: Picked off by @mac_bz! https://t.co/3fR8phhfhU 0.8217828128502452
I really like Gillislee. Been better than his stats would show so far. #patriots 0.6202311894619086
#Patriots formation is TIGHT, with both WRs close to the formation.  Says crossing routes and outside run.  Holding true to form 0.5009359808919097
RT @MikeReiss: Patriots sideline as national ant

RT @BenVolin: Gostkowski 58 FG is GOOD. Plenty of leg, going with the wind. Patriots pick up 3 key points. Panthers lead 17-16 at halftime 0.8633600402746519
RT @AshAlexiss: Disgraceful. This defense needs a Belichick reality check 🙄 #Patriots #CARvsNE 0.5552220203157937
RT @Patriots: Career-high 58-yarder: ✔ https://t.co/h6ooPuE0Wo 0.8122216935903996
RT @Patriots: Career-high 58-yarder: ✔ https://t.co/h6ooPuE0Wo 0.8122216935903996
Better to be to one point behind than 4. #GoPats @Patriots #Patriots 0.7222550654756835
Off the post and in by Gostkowski, needed that! #Patriots 0.47865993482646324
Gostkowski! Haha suck it Mazz #Patriots #Panthers #NFL 0.19909230224952704
HALFTIME

Panthers: 17
Patriots: 16

Live updates: https://t.co/Wrh8ojvLwP https://t.co/ZoZ7Uf3D8Q 0.6706128161822053
RT @Patriots: Career-high 58-yarder: ✔ https://t.co/h6ooPuE0Wo 0.8122216935903996
Career long 58-yard FG for SG3! Patriots down 17-16 heading into the locker room 0.42696442904151966
Stephen Gostkowski hit

Waterfall started w/ multiple communication errors, #Patriots D now gettin outplayed in too many 1-on-1 spots. Several people, not any 1 guy 0.4623373295666253
RT @Tashville401: @Patriots I am:

⚪️ a man
⚪️ a woman
🔘 the New England Patriots

seeking a:

⚪️ man
⚪️ woman
🔘 NFL caliber defense…  0.588442639453826
RT @ImJustAMel: Good morning Patriots🇺🇸

I love waking up to the sounds of liberal tears 

It's going to be 8 amazing yrs…  0.9454744412460299
Hey,Patriots' D.....WAKE UP! #NEvsCAR 0.8123259165389477
RT @Tashville401: @Patriots I am:

⚪️ a man
⚪️ a woman
🔘 the New England Patriots

seeking a:

⚪️ man
⚪️ woman
🔘 NFL caliber defense…  0.588442639453826
#Patriots still can't get a stop on the #Panthers, but at least the kick was no good. #CARvsNE #GoPats 0.30346523864926195
Alright offense, don’t squabble possession of the ball! #CARvsNE #GoPats @Patriots #Patriots 0.7132992012400966
Live look at the Patriots secondary trying to figure out who to cover https://t.co/5XzAVApABN 0.544

Time for them to kick the ball back to Mr. Brady. #CARvsNE #GoPats @Patriots #Patriots 0.6505044333597946
RT @Patriots: Welcome back, #boomtower! https://t.co/kST9R0TNTK 0.8122216935903996
RT @Patriots: Welcome back, #boomtower! https://t.co/kST9R0TNTK 0.8122216935903996
RT @Patriots: Welcome back, #boomtower! https://t.co/kST9R0TNTK 0.8122216935903996
Hightower brings down Newton. This is the first three and out of the game. #Patriots with a big forced punt. 0.49594609478259
RT @Patriots: Welcome back, #boomtower! https://t.co/kST9R0TNTK 0.8122216935903996
RT @Dan_Shaughnessy: Patriots have enough time to win by 10 points. #WatchOtherGuysChoke 0.6505044333597946
That was an excellent #Patriots defensive series when they needed it most.  Call the spade a spade... 0.24912646122370374
RT @Patriots: .@DionLewisRB with eyes for the end zone. #GoPats https://t.co/yz54MkbHJd 0.6225849925801458
So I guess we can confirm that the Patriots have at least one good player on defense 0.586889474278

@farffanntastico Preocupate por tu equipo y deja de vivir de los fracasos de los Patriots 0.7132992012400966
Patriots will still be in the AFC Championship but going on the road is going to suck 0.03949825878488817
RT @PatriotsInform: And the #Patriots lose to the officials once again under the guise of the Panthers. Seriously though defense needs a wa… 0.31887052108646646
Patriots are lucky not to be 1-3 0.4584312279120568
Finally, this is the most embarrassed I’ve ever been of a Patriots team. But hey, the big issue is whether they stand or sit for a song, eh? 0.8322737156808998
RT @Panthers: Dab on 'em @Patriots 😃 https://t.co/E3LLMpHuG2 0.8217828128502452
@Patriots “Great game Gilmore”....said no one 0.7483498571288267
RT @PatsMilitia: Patriots: 2-0

Stephon Gilmore: 0-2 https://t.co/KqeJP3Axfw 0.8122216935903996
Game 4 of the 2017-2018 #NewEnglandPatriots season. Carolina Panthers @ New England Patriots. Cam Newton Vs. Tom Brady 0.6544101962077042
RT @Patriots: Game time. #GoPats 

In [ ]:
df.get_values()

In [15]:
table = database['roll_tide']
df = tweet_processor.process_database(table)

In [16]:
from src.visualization import visualize_sentiment
from src.visualization import visualize_count
my_map = visualize_sentiment(df)
my_map